In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install OpenNMT-py 3.x
!pip3 install OpenNMT-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.8/262.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Open the folder where you saved your prepapred datasets from the first exercise
%cd /content/drive/MyDrive/Bn-Hi_translation_project/
!ls

/content/drive/MyDrive/Bn-Hi_translation_project
 config.yaml   dataset	 nmt   notebook  'Read Me.gdoc'   run


#Create the Training Configuration File
The following config file matches most of the recommended values for the Transformer model Vaswani et al., 2017. As the current dataset is small, we reduced the following values:

train_steps - for datasets with a few millions of sentences, consider using a value between 100000 and 200000, or more! Enabling the option early_stopping can help stop the training when there is no considerable improvement.
valid_steps - 10000 can be good if the value train_steps is big enough.
warmup_steps - obviously, its value must be less than train_steps. Try 4000 and 8000 values.
Refer to OpenNMT-py training parameters for more details. If you are interested in further explanation of the Transformer model, you can check this article, Illustrated Transformer.

In [ ]:
# Create the YAML configuration file
# On a regular machine, you can create it manually or with nano
# Note here we are using some smaller values because the dataset is small
# For larger datasets, consider increasing: train_steps, valid_steps, warmup_steps, save_checkpoint_steps, keep_checkpoint

config = '''# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: /content/drive/MyDrive/Bn-Hi_translation_project/dataset/src_subword.txt.train
        path_tgt: /content/drive/MyDrive/Bn-Hi_translation_project/dataset/tgt_subword.txt.train
        transforms: [filtertoolong]
    valid:
        path_src: /content/drive/MyDrive/Bn-Hi_translation_project/dataset/src_subword.txt.dev
        path_tgt: /content/drive/MyDrive/Bn-Hi_translation_project/dataset/tgt_subword.txt.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: /content/drive/MyDrive/Bn-Hi_translation_project/run/source.vocab
tgt_vocab: /content/drive/MyDrive/Bn-Hi_translation_project/run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
tgt_seq_length: 150

# Tokenization options
src_subword_model: /content/drive/MyDrive/Bn-Hi_translation_project/dataset/source.model
tgt_subword_model: /content/drive/MyDrive/Bn-Hi_translation_project/dataset/target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: /content/drive/MyDrive/Bn-Hi_translation_project/models/model.bnhi

# Resume training from the last checkpoint
#train_from: models/model.bnhi_step_XXXX.pt  # Replace XXXX with the last saved step number


# Stop training if it does not imporve after n validations
early_stopping: 4

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 1000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 3435

# Default: 100000 - Train the model to max n steps
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 10000

# Default: 10000 - Run validation after n steps
valid_steps: 1000

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 1000
report_every: 100

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 4096   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 2
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [ ]:
# [Optional] Check the content of the configuration file
!cat config.yaml

# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: /content/drive/MyDrive/Bn-Hi_translation_project/dataset/src_subword.txt.train
        path_tgt: /content/drive/MyDrive/Bn-Hi_translation_project/dataset/tgt_subword.txt.train
        transforms: [filtertoolong]
    valid:
        path_src: /content/drive/MyDrive/Bn-Hi_translation_project/dataset/src_subword.txt.dev
        path_tgt: /content/drive/MyDrive/Bn-Hi_translation_project/dataset/tgt_subword.txt.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: /content/drive/MyDrive/Bn-Hi_translation_project/run/source.vocab
tgt_vocab: /content/drive/MyDrive/Bn-Hi_translation_project/run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Token

#Build Vocabulary
For large datasets, it is not feasable to use all words/tokens found in the corpus. Instead, a specific set of vocabulary is extracted from the training dataset, usually betweeen 32k and 100k words. This is the main purpose of the vocabulary building step.

In [ ]:
# Find the number of CPUs/cores on the machine
!nproc --all

2


In [ ]:
# Build Vocabulary

# -config: path to your config.yaml file
# -n_sample: use -1 to build vocabulary on all the segment in the training dataset
# -num_threads: change it to match the number of CPUs to run it faster

!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 2

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2024-12-07 07:36:34,605 INFO] Counter vocab from -1 samples.
[2024-12-07 07:36:34,606 INFO] n_sample=-1: Build vocab on full datasets.
[2024-12-07 07:36:35,531 INFO] * Transform statistics for corpus_1(50.00%):
			* FilterTooLongStats(filtered=2)

[2024-12-07 07:36:35,547 INFO] * Transform statistics for corpus_1(50.00%):
			* FilterTooLongStats(filtered=4)

[2024-12-07 07:36:35,580 INFO] Counters src: 6221
[2024-12-07 07:36:35,580 INFO] Counters tgt: 6120


In [ ]:
# Check if the GPU is active
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# Check if the GPU is visable to PyTorch

import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

gpu_memory = torch.cuda.mem_get_info(0)
print("Free GPU memory:", gpu_memory[0]/1024**2, "out of:", gpu_memory[1]/1024**2)

False


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available")
    print("Device name:", torch.cuda.get_device_name(0))
    gpu_memory = torch.cuda.mem_get_info(0)
    print("Free GPU memory:", gpu_memory[0]/1024**2, "out of:", gpu_memory[1]/1024**2)
else:
    print("CUDA is not available. Please make sure you have an NVIDIA GPU and the driver installed.")

CUDA is not available. Please make sure you have an NVIDIA GPU and the driver installed.


#Training
Now, start training your NMT model! 🎉 🎉 🎉

In [ ]:
!rm -rf \
/content/drive/MyDrive/Bn-Hi_translation_project/models/

In [ ]:
# Train the NMT model
!onmt_train -config \
/content/drive/MyDrive/Bn-Hi_translation_project/config.yaml

Streaming output truncated to the last 5000 lines.
			* corpus_1: 4435
[2024-12-07 09:54:42,462 INFO] Weighted corpora loaded so far:
			* corpus_1: 4436
[2024-12-07 09:54:45,946 INFO] Weighted corpora loaded so far:
			* corpus_1: 4437
[2024-12-07 09:54:46,168 INFO] Weighted corpora loaded so far:
			* corpus_1: 4438
[2024-12-07 09:54:46,399 INFO] Weighted corpora loaded so far:
			* corpus_1: 4439
[2024-12-07 09:54:46,618 INFO] Weighted corpora loaded so far:
			* corpus_1: 4440
[2024-12-07 09:54:46,863 INFO] Weighted corpora loaded so far:
			* corpus_1: 4441
[2024-12-07 09:55:02,700 INFO] Step 3300/10000; acc: 98.8; ppl:   3.5; xent: 1.3; lr: 0.00154; sents:  108528; bsz: 2164/2442/271; 3933/4438 tok/s;   7183 sec;
[2024-12-07 09:55:06,689 INFO] * Transform statistics for corpus_1(100.00%):
			* FilterTooLongStats(filtered=108)

[2024-12-07 09:55:06,690 INFO] Weighted corpora loaded so far:
			* corpus_1: 4442
[2024-12-07 09:55:06,938 INFO] Weighted corpora loaded so far:
			* corp

In [ ]:
# For error debugging try:
# !dmesg -T

#Translation
Translation Options:

-model - specify the last model checkpoint name; try testing the quality of multiple checkpoints
-src - the subworded test dataset, source file
-output - give any file name to the new translation output file
-gpu - GPU ID, usually 0 if you have one GPU. Otherwise, it will translate on CPU, which would be slower.
-min_length - [optional] to avoid empty translations
-verbose - [optional] if you want to print translations

In [ ]:
# Translate the "subworded" source file of the test dataset
# Change the model name, if needed.
!onmt_translate \
-model /content/drive/MyDrive/Bn-Hi_translation_project/models/model.bnhi_step_1000.pt \
-src /content/drive/MyDrive/Bn-Hi_translation_project/dataset/src_subword.txt.test \
-output /content/drive/MyDrive/Bn-Hi_translation_project/dataset/bn-hi-translated.txt \
-gpu 0 -min_length 1


[2024-12-07 11:02:59,240 INFO] Loading checkpoint from /content/drive/MyDrive/Bn-Hi_translation_project/models/model.bnhi_step_1000.pt
[2024-12-07 11:03:00,732 INFO] Loading data into the model
[2024-12-07 11:03:17,122 INFO] PRED SCORE: -0.2838, PRED PPL: 1.33 NB SENTENCES: 3000
Time w/o python interpreter load/terminate:  17.90147876739502


In [ ]:
# Check the first 5 lines of the translation file
!head -n 5 /content/drive/MyDrive/Bn-Hi_translation_project/dataset/bn-hi-translated.txt

▁तंत्र ▁तयशुदा ▁का ▁उपयोग ▁करें ▁(_ U )
▁यह ▁ईमेल ▁पता ▁का ▁उपयोग ▁कर ▁रहा ▁है ▁और ▁आप को ▁क ्रैश ▁के ▁लिए ▁ यदि ▁आप ▁किस ी ▁तरह ▁इस े ▁जोड़न ा ▁चाहत े ▁हैं .
▁डिस्क ▁को ▁खुला ▁छोड़ें ▁अन्य ▁फाइल ों ▁को ▁जोड़ ने ▁के ▁लिए ▁(_ o )
▁संपर्क ▁पूर्वावलोकन ▁विंडो ▁में ▁नक्शे ▁दिखाएँ
▁संपर्क ▁सूची ▁छाँट ▁मापदंड


In [ ]:
# If needed install/update sentencepiece
!pip3 install --upgrade -q sentencepiece

# Desubword the translation file
!python3 \
/content/drive/MyDrive/Bn-Hi_translation_project/nmt/MT-Preparation/subwording/3-desubword.py \
/content/drive/MyDrive/Bn-Hi_translation_project/nmt/target.model \
/content/drive/MyDrive/Bn-Hi_translation_project/dataset/bn-hi-translated.txt

Done desubwording! Output: /content/drive/MyDrive/Bn-Hi_translation_project/dataset/bn-hi-translated.txt.desubword


In [ ]:
# Check the first 5 lines of the desubworded translation file
!head -n 5 \
/content/drive/MyDrive/Bn-Hi_translation_project/dataset/bn-hi-translated.txt.desubword

तंत्र तयशुदा का उपयोग करें (_U)
यह ईमेल पता का उपयोग कर रहा है और आपको क्रैश के लिए यदि आप किसी तरह इसे जोड़ना चाहते हैं.
डिस्क को खुला छोड़ें अन्य फाइलों को जोड़ने के लिए (_o)
संपर्क पूर्वावलोकन विंडो में नक्शे दिखाएँ
संपर्क सूची छाँट मापदंड


In [ ]:
# Desubword the target file (reference) of the test dataset
# Note: You might as well have split files *before* subwording during dataset preperation,
# but sometimes datasets have tokeniztion issues, so this way you are sure the file is really untokenized.
!python3 \
/content/drive/MyDrive/Bn-Hi_translation_project/nmt/MT-Preparation/subwording/3-desubword.py \
/content/drive/MyDrive/Bn-Hi_translation_project/nmt/target.model \
/content/drive/MyDrive/Bn-Hi_translation_project/dataset/tgt_subword.txt.test


Done desubwording! Output: /content/drive/MyDrive/Bn-Hi_translation_project/dataset/tgt_subword.txt.test.desubword


In [ ]:
# Check the first 5 lines of the desubworded reference
!head -n 5 \
/content/drive/MyDrive/Bn-Hi_translation_project/dataset/tgt_subword.txt.test.desubword

लोकेल तयशुदा का उपयोग करें
ईमेल अनुप्रयोग का उपयोग करने के लिए आपको एक खाते को स्थापित करने की आवश्यकता होगी. नीचे अपना ईमेल पता और कूटशब्द डाले और हमलोग कोशिश करेगें और सभी सेटिंग को संपन्न करेगें. अगर हम इसे स्वचालित रूप से नहीं कर सकते हैं तो आपको अच्छी तरह से अपने सर्वर की जानकारी की आवश्यकता होगी.
डिस्क को खुला छोड़ें अन्य फाइलों को जोड़ने के लिए (_o)
संपर्क पूर्वावलोकन विंडो दिखाएँ
सूची छाँट मापदंड से संपर्क करें


#MT Evaluation
There are several MT Evaluation metrics such as BLEU, TER, METEOR, COMET, BERTScore, among others.

Here we are using BLEU. Files must be detokenized/desubworded beforehand.

In [ ]:
# Download the BLEU script
!wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py

--2024-12-07 11:11:26--  https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957 [text/plain]
Saving to: ‘compute-bleu.py’

compute-bleu.py     100%[===================>]     957  --.-KB/s    in 0s      

2024-12-07 11:11:26 (46.2 MB/s) - ‘compute-bleu.py’ saved [957/957]



In [ ]:
# Install sacrebleu
!pip3 install sacrebleu

In [ ]:
# Evaluate the translation (without subwording)
!python3 compute-bleu.py \
/content/drive/MyDrive/Bn-Hi_translation_project/dataset/tgt_subword.txt.test.desubword \
/content/drive/MyDrive/Bn-Hi_translation_project/dataset/bn-hi-translated.txt.desubword


Reference 1st sentence: लोकेल तयशुदा का उपयोग करें
MTed 1st sentence: तंत्र तयशुदा का उपयोग करें (_U)
BLEU:  43.71157837181612


#Date 10/12/2024

In [1]:
!pip3 install ctranslate2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 50.5 MB/s eta 0:00:00


In [5]:
# • OpenNMT-py:
!ct2-opennmt-py-converter \
--model_path /content/drive/MyDrive/Bn-Hi_translation_project/models/model.bnhi_step_1000_fixed.pt \
--output_dir /content/drive/MyDrive/Bn-Hi_translation_project/bnhi_ctranslate2/ \
--quantization int8


/usr/local/lib/python3.10/dist-packages/ctranslate2/converters/opennmt_py.py:177: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(self._model_path, map

In [7]:
# First convert your OpenNMT-py or OpenNMT-tf model to a CTranslate2 model.
# pip3 install ctranslate2
# • OpenNMT-py:
# ct2-opennmt-py-converter --model_path model.pt --output_dir enja_ctranslate2 --quantization int8
# • OpenNMT-tf:
# ct2-opennmt-tf-converter --model_path model --output_dir enja_ctranslate2 --src_vocab source.vocab --tgt_vocab target.vocab --model_type TransformerBase --quantization int8


import ctranslate2
import sentencepiece as spm

# Set file paths
source_file_path = "/content/drive/MyDrive/Bn-Hi_translation_project/dataset/src_filter.txt"
target_file_path = "/content/drive/MyDrive/Bn-Hi_translation_project/dataset/src_filter_translated.txt"

sp_source_model_path = "/content/drive/MyDrive/Bn-Hi_translation_project/dataset/source.model"
sp_target_model_path = "/content/drive/MyDrive/Bn-Hi_translation_project/dataset/target.model"

ct_model_path = "/content/drive/MyDrive/Bn-Hi_translation_project/bnhi_ctranslate2/"


# Load the source SentecePiece model
sp = spm.SentencePieceProcessor()
sp.load(sp_source_model_path)

# Open the source file
with open(source_file_path, "r") as source:
  lines = source.readlines()

source_sents = [line.strip() for line in lines]

# Subword the source sentences
source_sents_subworded = sp.encode_as_pieces(source_sents)

# Translate the source sentences
translator = ctranslate2.Translator(ct_model_path, device="cpu")  # or "cuda" for GPU
translations = translator.translate_batch(source_sents_subworded, batch_type="tokens", max_batch_size=4096)
translations = [translation.hypotheses[0] for translation in translations]

# Load the target SentecePiece model
sp.load(sp_target_model_path)

# Desubword the target sentences
translations_desubword = sp.decode(translations)


# Save the translations to the a file
with open(target_file_path, "w+", encoding="utf-8") as target:
  for line in translations_desubword:
    target.write(line.strip() + "\n")

print("Done")

Done


In [8]:
# First convert your OpenNMT-py or OpenNMT-tf model to a CTranslate2 model.
# pip3 install ctranslate2
# • OpenNMT-py:
# ct2-opennmt-py-converter --model_path model.pt --output_dir enja_ctranslate2 --quantization int8
# • OpenNMT-tf:
# ct2-opennmt-tf-converter --model_path model --output_dir enja_ctranslate2 --src_vocab source.vocab --tgt_vocab target.vocab --model_type TransformerBase --quantization int8


import ctranslate2
import sentencepiece as spm

# Set file paths
source_file_path = "/content/drive/MyDrive/Bn-Hi_translation_project/dataset/sayak.txt"
target_file_path = "/content/drive/MyDrive/Bn-Hi_translation_project/dataset/sayak_translated.txt"

sp_source_model_path = "/content/drive/MyDrive/Bn-Hi_translation_project/dataset/source.model"
sp_target_model_path = "/content/drive/MyDrive/Bn-Hi_translation_project/dataset/target.model"

ct_model_path = "/content/drive/MyDrive/Bn-Hi_translation_project/bnhi_ctranslate2/"



# Load the source SentecePiece model
sp = spm.SentencePieceProcessor()
sp.load(sp_source_model_path)

# Open the source file
with open(source_file_path, "r") as source:
  lines = source.readlines()

source_sents = [line.strip() for line in lines]

# Subword the source sentences
source_sents_subworded = sp.encode_as_pieces(source_sents)

# Translate the source sentences
translator = ctranslate2.Translator(ct_model_path, device="cpu")  # or "cuda" for GPU
translations = translator.translate_batch(source_sents_subworded, batch_type="tokens", max_batch_size=4096)
translations = [translation.hypotheses[0] for translation in translations]

# Load the target SentecePiece model
sp.load(sp_target_model_path)

# Desubword the target sentences
translations_desubword = sp.decode(translations)


# Save the translations to the a file
with open(target_file_path, "w+", encoding="utf-8") as target:
  for line in translations_desubword:
    target.write(line.strip() + "\n")

print("Done")

Done
